## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [3]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c



float64
8.69 ms ± 49.5 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
19.3 ms ± 76 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
19.5 ms ± 75 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [1]:
!nvidia-smi

Tue Dec 23 12:09:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080        Off |   00000000:01:00.0 Off |                  N/A |
| 31%   43C    P0             38W /  180W |       0MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
#3.2

import cupy as cp
from cupyx.profiler import benchmark

# Copia explícita de los datos desde CPU a GPU
x_gpu = cp.asarray(a_cpu)
y_gpu = cp.asarray(b_cpu)

def grade2_cupy(x, y, a, b, c):
    return a * x**2 + b * y + c

# Benchmark en GPU (incluye copia previa)
res = benchmark(
    grade2_cupy,
    args=(x_gpu, y_gpu, a, b, c),
    n_repeat=20
)

print(f"Tiempo CuPy con copia CPU->GPU: {cp.mean(res.gpu_times):.4f} s")

Tiempo CuPy con copia CPU->GPU: 0.0055 s


In [5]:
# Creación directa de los datos en GPU (sin copia desde CPU)
x_gpu = cp.random.rand(size, dtype=cp.float32)
y_gpu = cp.random.rand(size, dtype=cp.float32)

res = benchmark(
    grade2_cupy,
    args=(x_gpu, y_gpu, a, b, c),
    n_repeat=20
)

print(f"Tiempo CuPy sin copia CPU->GPU: {cp.mean(res.gpu_times):.4f} s")

Tiempo CuPy sin copia CPU->GPU: 0.0009 s


In [6]:
#3.2b

from numba import vectorize

@vectorize(['float32(float32, float32, float32, float32, float32)'],
           target='cuda')
def grade2_numba(x, y, a, b, c):
    return a * x * x + b * y + c

In [7]:
# Datos en CPU (NumPy)
x_cpu = a_cpu.astype(np.float32)
y_cpu = b_cpu.astype(np.float32)

res = benchmark(
    grade2_numba,
    args=(x_cpu, y_cpu, np.float32(a), np.float32(b), np.float32(c)),
    n_repeat=20
)

print(f"Tiempo Numba GPU con copia implícita: {cp.mean(res.gpu_times):.4f} s")

Tiempo Numba GPU con copia implícita: 0.0090 s


In [8]:
from numba import cuda

# Copia manual de datos a GPU
x_dev = cuda.to_device(x_cpu)
y_dev = cuda.to_device(y_cpu)

res = benchmark(
    grade2_numba,
    args=(x_dev, y_dev, np.float32(a), np.float32(b), np.float32(c)),
    n_repeat=20
)

print(f"Tiempo Numba GPU sin contar copia: {cp.mean(res.gpu_times):.4f} s")

Tiempo Numba GPU sin contar copia: 0.0015 s


Al ejecutar el cálculo en la GPU se observa una reducción muy clara del tiempo de ejecución respecto a la versión en CPU. No obstante, la ganancia obtenida depende en gran medida de cómo se gestionan los datos.

En el caso en el que los arrays se copian desde la CPU a la GPU, el cálculo en sí es muy rápido, pero existe un coste adicional asociado a la transferencia de memoria entre ambos dispositivos. Por el contrario, cuando los datos se crean directamente en la GPU, se evita este paso y el tiempo total se reduce de forma significativa.

Estos resultados muestran que, en aplicaciones aceleradas por GPU, no solo es importante el cálculo, sino también minimizar las transferencias de datos entre CPU y GPU para obtener el máximo rendimiento.

La ejecución en GPU con Numba depende en gran medida del coste de transferencia de datos entre CPU y GPU. Cuando la copia se realiza de forma automática desde arrays en CPU, el tiempo total aumenta. Al copiar los datos previamente a la GPU, se mide únicamente el tiempo de cómputo, obteniéndose un rendimiento claramente superior.